<a href="https://colab.research.google.com/github/Bag0niku/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create the environment for the analysis

In [2]:
# Install Spark and Java
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
import os
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# the SQL database we wil be using is Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

!pip install -q findspark # dependency to find the pyspark installation
import findspark  
findspark.init()  # initialize the search for pyspark

# Import remaining dependencies
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
from getpass import getpass
import pandas as pd

# database password
password = getpass()

# location of the file with the data of amazon reviews
url = "https://videogames-review-analysis.s3.us-west-2.amazonaws.com/amazon_reviews_us_Video_Games_v1_00.tsv"
file = "amazon_reviews_us_Video_Games_v1_00.tsv"


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done
--2022-10-05 06:12:01--  https://jdbc.postgresql.or

In [3]:
spark = SparkSession.builder.appName("AmazonReviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazon-vine-analysis.comqyjfkggk2.us-west-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

# retrieve the data file
spark.sparkContext.addFile(url)

# Deliverable 1: Insert data into AWS Database

In [4]:

## read the file into a dataframe
df = spark.read.csv(SparkFiles.get(file), sep="\t", header=True, inferSchema=True)

# modify the review_date to only show the date, not time.
df = df.withColumnRenamed("review_date", "datetime")
df = df.withColumn("review_date", to_date(df["datetime"]))

# create a SQL table name for the DataFrme
df.createOrReplaceTempView("big_table")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|           datetime|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|

In [11]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- review_date: date (nullable = true)



In [19]:
review_id_table = spark.sql("SELECT review_id, customer_id, product_id, product_parent, review_date FROM big_table;")
review_id_table.createOrReplaceTempView("review_id_table")
review_id_table.show(10)


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [20]:
# Products table
products_table = spark.sql("SELECT DISTINCT(product_id), product_title FROM big_table;")
products_table.createOrReplaceTempView("products_table")
products_table.show(10, truncate=False)

+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                                                                                           |
+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|B00CJ7IUI6|The Elder Scrolls V: Skyrim Legendary Edition                                                                                                                                                                           |
|B00DHF39KS|Wolfenstein: The New Order                                          

In [22]:
# count of reviews by each customer
customers_table = spark.sql("SELECT customer_id, COUNT(review_id) AS customer_count FROM big_table GROUP BY customer_id;")
customers_table.createOrReplaceGlobalTempView("customers_table")
customers_table.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
+-----------+--------------+
only showing top 10 rows



In [25]:
# vine_table
vine_table = spark.sql("SELECT review_id, star_rating, helpful_votes, total_votes, vine, verified_purchase FROM big_table;")
vine_table.createOrReplaceTempView("vine_table")
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [16]:
# write the newly created tables (vine_table, customers_table, products_table, review_id_table) to postgresSQL Database in AWS

# test the first one and check it
vine_table.write.jdbc(url=jdbc_url, mode=mode, table="vine_table", properties=config)

In [14]:
# check to make sure the SQL INSERT is worked
test_db_write = spark.read.jdbc(url=jdbc_url, table="vine_table", properties=config)
test_db_write.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

# Deliverable 2: 

In [17]:
# write the remaining 3 new tables to the SQL Database on AWS
customers_table.write.jdbc(url=jdbc_url, mode=mode, table="customers_table", properties=config)
products_table.write.jdbc(url=jdbc_url, mode=mode, table="products_table", properties=config)
review_id_table.write.jdbc(url=jdbc_url, mode=mode, table="review_id_table", properties=config)

In [27]:
spark.sql("select * from vine_table limit 5;").show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+

